In [119]:
from py5paisa import FivePaisaClient
from py5paisa.order import Order, OrderType, Exchange, Bo_co_order
import time
import math
import pandas as pd
import warnings
from datetime import datetime

In [120]:
warnings.filterwarnings("ignore", category=FutureWarning)

In [121]:
cred = {
    "APP_NAME": "5P56409084",
    "APP_SOURCE": "9217",
    "USER_ID": "pybeQPEbjju",
    "PASSWORD": "AESDZQGqyAa",
    "USER_KEY": "542GTEZguBbLERUXRZe26rLvfWU2X2KD",
    "ENCRYPTION_KEY": "l8sRTIsjl1JhUrz8X48XQJzKGI4k0gft",
}
client = FivePaisaClient(
    email="vasuappliedai@gmail.com", passwd="Vasudeva@1", dob="19981218", cred=cred
)
client.login()

 00:08:09 | Logged in!!


In [116]:
def get_current_expiry_strikes(symbol, start_strike, end_strike):
    temp_df = script_df["FullName"].str.contains(
        pat=symbol + " CE", regex=True)
    call_df = script_df[temp_df]
    call_df = call_df[call_df["StrikeRate"] > start_strike]
    call_df = call_df[call_df["StrikeRate"] < end_strike]
    call_df.to_csv("bank_nifty_call_expiry.csv")
    temp_df = script_df["FullName"].str.contains(
        pat=symbol + " PE", regex=True)
    put_df = script_df[temp_df]
    put_df = put_df[put_df["StrikeRate"] > start_strike]
    put_df = put_df[put_df["StrikeRate"] < end_strike]
    put_df.to_csv("bank_nifty_put_expiry.csv")
    return call_df, put_df

def map_prices_to_strike_prices(data_frame, entry_point=None):
    temp_rows = []
    for _, row in data_frame.iterrows():
        script_code = row["Scripcode"]
        try:
            if entry_point is None:
                close_point = client.historical_data("N", "D", script_code, "5m", 
                    given_date, given_date).tail(1)
                temp_rows.append(close_point["Close"].iloc[0])
            else:
                close_points = client.historical_data("N", "D", script_code, "1m", 
                    given_date, given_date)
                if close_points is not None:
                    close_points["Datetime"] = pd.to_datetime(close_points["Datetime"])
                    close_point = close_points[close_points["Datetime"]==datetime.fromisoformat(str(entry_point))]["Close"].iloc[0]
                    temp_rows.append(close_point)
                else: 
                    temp_rows.append(9999)
        except:
            temp_rows.append(9999)
            continue
    data_frame["LTP"] = temp_rows
    return data_frame.sort_values(by=["LTP"])


def adjust_strangle(option_type,desired_price,entry_point):
    call_df, put_df = get_current_expiry_strikes(bank_nifty_symbol, strikePrice*0.9, strikePrice*1.1)
    if option_type == "CE":
        call_option_chain = map_prices_to_strike_prices(call_df,entry_point)
        call_script_code = call_option_chain[call_option_chain["LTP"]<desired_price].tail(1)["Scripcode"].iloc[0]
        call_sell = call_option_chain[call_option_chain["LTP"]<desired_price].tail(1)["LTP"].iloc[0]
        call_symbol = call_option_chain[call_option_chain["LTP"]<desired_price].tail(1)["Name"].iloc[0]
        print("Sold call call_sell, call_symbol, call_script_code ",call_sell,call_symbol, call_script_code)
    if option_type == "PE":
        put_option_chain = map_prices_to_strike_prices(put_df,entry_point)
        desired_put_df = put_option_chain[put_option_chain["LTP"]<desired_price].tail(1)
        put_script_code = desired_put_df["Scripcode"].iloc[0]
        put_sell = desired_put_df["LTP"].iloc[0]
        put_symbol = desired_put_df["Name"].iloc[0]
        print("Sold put sell put_sell, put_symbol, put_script_code ",put_sell,put_symbol,put_script_code)


def start_strangle(entry_time,call_script_code,put_script_code):
    call_df = client.historical_data('N','D',call_script_code,'1m',given_date,given_date)
    call_df["Datetime"] = pd.to_datetime(call_df["Datetime"])
    call_df["Close CE"] = call_df["Close"]
    call_df = call_df[call_df["Datetime"]>datetime.fromisoformat(entry_time)]
    call_df = call_df[["Datetime","Close CE"]]
    put_df = client.historical_data('N','D',put_script_code,'1m',given_date,given_date)
    put_df["Datetime"] = pd.to_datetime(put_df["Datetime"])
    put_df["Close PE"] = put_df["Close"]
    put_df = put_df[put_df["Datetime"]>datetime.fromisoformat(entry_time)]
    put_df = put_df[["Datetime","Close PE"]]
    merged_df = pd.merge(call_df,put_df,on="Datetime")
    for index,row in merged_df.iterrows() : 
        call_point = row["Close CE"]
        put_point = row["Close PE"]
        if(call_point*0.5 > put_point):
            print("PE is half of CE at time, call_price and put_price ",row["Datetime"],call_point,put_point)
            adjust_strangle("PE",call_point*0.8,row["Datetime"])
            return "PE",row["Datetime"]
        elif(call_point < put_point*0.5):        
            print("CE is half of PE at index, call_price and put_price ",row["Datetime"],call_point,put_point)
            adjust_strangle("CE",put_point*0.8,row["Datetime"])
            return "CE",row["Datetime"]

       
def rounded_number(number):
    return (round(strikePrice / 1000, 1) * 1000).astype(int)

In [11]:
# This code should be executed on Thursday or when profit is booked
nifty_script_code = 999920000
banknifty_script_code = 999920005
given_date = datetime.today().strftime("%Y-%m-%d")
script_df = pd.read_csv("scripmaster-csv-format.csv",low_memory=False)
bank_nifty_symbol = "BANKNIFTY 07 Apr 2022"
strikePrice = client.historical_data("N", "C", banknifty_script_code, "1m", given_date, given_date)["Open"].iloc[240]
strikePrice = rounded_number(strikePrice)
strikePrice

35700

In [32]:
call_df, put_df = get_current_expiry_strikes(bank_nifty_symbol, strikePrice*0.75, strikePrice*1.25)
call_df,put_df

(       Exch ExchType  Scripcode                               Name Series  \
 43517     N        D      38138  BANKNIFTY 07 Apr 2022 CE 30800.00     XX   
 48618     N        D      47283  BANKNIFTY 07 Apr 2022 CE 28900.00     XX   
 48620     N        D      47285  BANKNIFTY 07 Apr 2022 CE 29000.00     XX   
 48629     N        D      47294  BANKNIFTY 07 Apr 2022 CE 29100.00     XX   
 48631     N        D      47296  BANKNIFTY 07 Apr 2022 CE 29200.00     XX   
 ...     ...      ...        ...                                ...    ...   
 113185    N        D      42217  BANKNIFTY 07 Apr 2022 CE 38900.00     XX   
 114296    N        D      43756  BANKNIFTY 07 Apr 2022 CE 39000.00     XX   
 114329    N        D      43789  BANKNIFTY 07 Apr 2022 CE 39100.00     XX   
 114332    N        D      43792  BANKNIFTY 07 Apr 2022 CE 39200.00     XX   
 114334    N        D      43794  BANKNIFTY 07 Apr 2022 CE 39300.00     XX   
 
                      Expiry CpType  StrikeRate WireCat       

In [34]:
call_option_chain = map_prices_to_strike_prices(call_df,240)
put_option_chain = map_prices_to_strike_prices(put_df,240)

 22:20:15 | Length mismatch: Expected axis has 0 elements, new values have 6 elements
 22:20:15 | Length mismatch: Expected axis has 0 elements, new values have 6 elements
 22:20:15 | Length mismatch: Expected axis has 0 elements, new values have 6 elements
 22:20:15 | Length mismatch: Expected axis has 0 elements, new values have 6 elements
 22:20:16 | Length mismatch: Expected axis has 0 elements, new values have 6 elements
 22:20:16 | Length mismatch: Expected axis has 0 elements, new values have 6 elements
 22:20:16 | Length mismatch: Expected axis has 0 elements, new values have 6 elements
 22:20:16 | Length mismatch: Expected axis has 0 elements, new values have 6 elements
 22:20:16 | Length mismatch: Expected axis has 0 elements, new values have 6 elements
 22:20:16 | Length mismatch: Expected axis has 0 elements, new values have 6 elements
 22:20:16 | Length mismatch: Expected axis has 0 elements, new values have 6 elements
 22:20:16 | Length mismatch: Expected axis has 0 eleme

In [ ]:
import pickle
# option_chains = (call_option_chain,put_option_chain)
# with open("option_chain.pickle","wb") as file:
#     pickle.dump(option_chains,file)

file = open("option_chain.pickle","rb")
call_option_chain,put_option_chain = pickle.load(file)

In [122]:
desired_call_price = 115
desired_put_price = 115

In [36]:
call_option_chain[call_option_chain["LTP"]<desired_call_price].tail(1)

,Exch,ExchType,Scripcode,Name,Series,Expiry,CpType,StrikeRate,WireCat,ISIN,FullName,LTP
113144,N,D,42176,BANKNIFTY 07 Apr 2022 CE 37700.00,XX,2022-04-07 14:30:00,CE,37700.0,,BANKNIFTY,BANKNIFTY 07 Apr 2022 CE 37700.00,104.1


In [37]:
put_option_chain[put_option_chain["LTP"]<desired_put_price].tail(1)

,Exch,ExchType,Scripcode,Name,Series,Expiry,CpType,StrikeRate,WireCat,ISIN,FullName,LTP
111937,N,D,41909,BANKNIFTY 07 Apr 2022 PE 33400.00,XX,2022-04-07 14:30:00,PE,33400.0,,BANKNIFTY,BANKNIFTY 07 Apr 2022 PE 33400.00,101.35


In [38]:
call_script_code = call_option_chain[call_option_chain["LTP"]<desired_call_price].tail(1)["Scripcode"].iloc[0]
put_script_code = put_option_chain[put_option_chain["LTP"]<desired_put_price].tail(1)["Scripcode"].iloc[0]
call_script_code,put_script_code

(42176, 41909)

In [123]:
call_script_code = 42176
put_script_code = 41934


In [164]:
start_strangle("2022-03-29T13:20:00",call_script_code,put_script_code)

PE is half of CE at time, call_price and put_price  2022-03-29 15:27:00 127.3 61.9
 00:54:54 | Length mismatch: Expected axis has 0 elements, new values have 6 elements
 00:54:54 | Length mismatch: Expected axis has 0 elements, new values have 6 elements
 00:54:55 | Length mismatch: Expected axis has 0 elements, new values have 6 elements
 00:54:55 | Length mismatch: Expected axis has 0 elements, new values have 6 elements
 00:54:55 | Length mismatch: Expected axis has 0 elements, new values have 6 elements
 00:54:55 | Length mismatch: Expected axis has 0 elements, new values have 6 elements
 00:54:56 | Length mismatch: Expected axis has 0 elements, new values have 6 elements
 00:54:56 | Length mismatch: Expected axis has 0 elements, new values have 6 elements
 00:54:56 | Length mismatch: Expected axis has 0 elements, new values have 6 elements
 00:54:56 | Length mismatch: Expected axis has 0 elements, new values have 6 elements
 00:54:56 | Length mismatch: Expected axis has 0 elements

('PE', Timestamp('2022-03-29 15:27:00'))

In [79]:
client.historical_data('N','D',call_script_code,'1m',given_date,given_date)["Close"].tail(1)
client.historical_data('N','D',put_script_code,'1m',given_date,given_date)["Close"].tail(1)

319    60.1
Name: Close, dtype: float64

In [ ]:
def day_close_check(call_script_code,put_script_code):
    call_point = client.historical_data('N','D',call_script_code,'1m',given_date,given_date)["Close"].iloc[370]
    put_point = client.historical_data('N','D',put_script_code,'1m',given_date,given_date)["Close"].iloc[370]
    if(call_point < put_point*0.75):
        print("Buy CE at ", call_point)
        print("Buy CE and sell CE near to put strike price")
        adjust_strangle("CE",put_point*0.8,370)
    if(put_point < call_point*0.75):
        print("Buy PE at ", put_point)
        print("selling PE near to call strike price")
        adjust_strangle("PE",call_point*0.8,370)
    
day_close_check(call_script_code,put_script_code)